In [1]:
from skylens import *

In [2]:
from dask.distributed import Lock

In [3]:
import faulthandler; faulthandler.enable()

In [4]:
import scipy.stats
import qmcpy

In [5]:
import multiprocessing
ncpu=multiprocessing.cpu_count()-1
vmem=psutil.virtual_memory()
mem=str(vmem.total/(1024**3)*0.95)+'GB'

In [6]:
from distributed import LocalCluster
from dask.distributed import Client  # we already had this above
#http://distributed.readthedocs.io/en/latest/_modules/distributed/worker.html
ncpu=5 #multiprocessing.cpu_count()-1
LC,scheduler_info=start_client(ncpu=None,n_workers=ncpu,threads_per_worker=1,
                                  memory_limit='120gb',dashboard_address=8801,processes=True)
client=client_get(scheduler_info=scheduler_info)
print('client: ',client)#,dask_dir,scheduler_info)


Start_client: No scheduler file, will start local cluster at  ./temp_skylens/pid26414/
client:  <Client: 'tcp://127.0.0.1:45743' processes=5 threads=5, memory=600.00 GB>


In [7]:
wigner_files={}
# wigner_files[0]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_0_reorder.zarr'
# wigner_files[2]= '/Users/Deep/dask_temp/dask_wig3j_l3500_w2100_2_reorder.zarr'
wig_home='/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/temp/'
wigner_files[0]= wig_home+'dask_wig3j_l3500_w2100_0_reorder.zarr'
wigner_files[2]= wig_home+'/dask_wig3j_l3500_w2100_2_reorder.zarr'


In [8]:
#setup parameters
bin_cl=True
lmax_cl=200
lmin_cl=2
l0=np.arange(lmin_cl,lmax_cl)

lmin_cl_Bins=lmin_cl+10
lmax_cl_Bins=lmax_cl-10
Nl_bins=25
l_bins=np.int64(np.logspace(np.log10(lmin_cl_Bins),np.log10(lmax_cl_Bins),Nl_bins+1))
lb=np.sqrt(l_bins[1:]*l_bins[:-1])

l=np.unique(np.int64(np.logspace(np.log10(lmin_cl),np.log10(lmax_cl),Nl_bins*20))) #if we want to use fewer ell
bin_cl=True

In [9]:
xi_win_approx=True
bin_xi=True
xi_SN_analytical=True
th_min=25/60
th_max=250./60
n_th_bins=20
th_bins=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins+1)
th=np.logspace(np.log10(th_min),np.log10(th_max),n_th_bins*40)
thb=np.sqrt(th_bins[1:]*th_bins[:-1])

In [10]:
WT=None

In [ ]:
do_pseudo_cl=True
do_xi=False

In [ ]:
if do_xi:
    WT_kwargs={'l':l0,'theta':th*d2r,'s1_s2':[(2,2),(2,-2),(0,0),(0,2)]}
    WT=wigner_transform(wig_d_taper_order_low=6,wig_d_taper_order_high=8,**WT_kwargs)


In [11]:
do_cov=True

SSV_cov=False
tidal_SSV_cov=False
Tri_cov=True

In [12]:
use_window=False
store_win=False

nside=32
window_lmax=nside

In [13]:
nzbins=2
zs_bin=lsst_source_tomo_bins(nbins=nzbins,use_window=use_window,nside=nside)

ns0:  27.0
[0.1   0.101 0.102 ... 2.998 2.999 3.   ] 100 5
source_tomo_bins, zmax 5.0 2.999799959991998 0.5999699939987997
Tracer utils has no tracers


In [14]:
do_cov=True

In [15]:
SSV_cov=False
bin_cl=True
do_cov=True
Tri_cov=False

In [17]:
use_binned_l=True
use_binned_theta=use_binned_l

In [18]:
corr_ggl=('galaxy','shear')
corr_gg=('galaxy','galaxy')
corr_ll=('shear','shear')
corrs=[corr_ll,corr_ggl,corr_gg]


In [19]:
power_spectra_kwargs={} #{'pk_lock':lock}

In [20]:
#use all ell
kappa0=Skylens(shear_zbins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, galaxy_zbins=zs_bin,
               use_window=use_window,Tri_cov=Tri_cov,
               use_binned_l=False,wigner_files=wigner_files,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
               store_win=store_win,window_lmax=window_lmax,
               sparse_cov=True,corrs=corrs,
               do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                use_binned_theta=False,
                nz_PS=10,do_pseudo_cl=do_pseudo_cl,xi_win_approx=True,
                xi_SN_analytical=xi_SN_analytical,#power_spectra_kwargs=power_spectra_kwargs
               )

pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 19 198 33.0 10
Window done. Size: 0.0


/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/cov_utils.py:91


In [21]:
if do_xi:
    print('doing xi')
    xi0G=kappa0.xi_tomo()
    xi_cov=client.compute(xi0G['stack']).result()
    cov_inv=np.linalg.inv(xi_cov['cov'].todense())
    data=xi_cov['xi']
else:
    print('doing cl')
    cl0G=kappa0.cl_tomo()
    cl_cov=client.compute(cl0G['stack']).result()
    cov_inv=np.linalg.inv(cl_cov['cov'].todense())
    data=cl_cov['pcl_b']

doing cl


/verafs/scratch/phy200040p/sukhdeep/miniconda3/lib/python3.8/site-packages/skylens-0.1-py3.8.egg/skylens/angular_power_spectra.py:115


scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary
cl cov graph done


In [22]:

print('Got data and cov')

Got data and cov


In [23]:
kappa0.do_cov=False

In [24]:
kappa0=Skylens(shear_zbins=zs_bin,do_cov=do_cov,bin_cl=bin_cl,l_bins=l_bins,l=l0, galaxy_zbins=zs_bin,
               use_window=use_window,Tri_cov=Tri_cov,
               use_binned_l=use_binned_l,wigner_files=wigner_files,
               SSV_cov=SSV_cov,tidal_SSV_cov=tidal_SSV_cov,f_sky=0.35,
               store_win=store_win,window_lmax=window_lmax,
               sparse_cov=True,corrs=corrs,
               do_xi=do_xi,bin_xi=bin_xi,theta_bins=th_bins,WT=WT,
                use_binned_theta=use_binned_theta,
                nz_PS=10,do_pseudo_cl=do_pseudo_cl,xi_win_approx=True,
                xi_SN_analytical=xi_SN_analytical,#power_spectra_kwargs=power_spectra_kwargs,
               Win=kappa0.Win
               )

pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 19 198 33.0 10
Window done. Size: 0.0
pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 2 25 33.0 10
Window done. Size: 0.0
scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary
scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary
set binned measure done
pk_params dict was none, intialized with default
cosmo_params dict was none, intialized with default
cosmology interpolation range 0.0 5.015
power spectra camb_pk_too_many_z
Win gen: step size 19 198 33.0 10
scatter_dict got empty dictionary
cl graph done
gather_dict got empty dictionary
scatter_dict got empty dictionary
cl graph done
gather_dict got e

In [25]:
params_order=['b1_0','b1_1']#,'Ase9','Om']
ndim=len(params_order)
priors_mean=[]#,10,.5]
priors_cov=np.eye(ndim)*0.5

cosmo_fid=kappa0.Ang_PS.PS.cosmo_params
Ang_PS=kappa0.Ang_PS


In [41]:
np.diag([1,1])

array([[1, 0],
       [0, 1]])

In [26]:
zs_bin1=copy.deepcopy(zs_bin)
del_k=['window','window_cl']
for k in del_k:
    if zs_bin1.get(k) is not None:
        del zs_bin1[k]
    for i in np.arange(zs_bin1['n_bins']):
        if zs_bin1[i].get(k) is not None:
            del zs_bin1[i][k]

In [27]:
def get_priors(params):#assume flat priors for now
    p=scipy.stats.multivariate_normal.pdf(params,mean=priors_mean,cov=priors_cov)
#     x=np.all(params>priors_max,axis=1)
#     x*=np.all(params<priors_max,axis=1)
#     p=np.zeros(len(params))
#     p[x]=-np.inf
    return p

In [28]:
def assign_zparams(zbins={},p_name='',p_value=None):
    pp=p_name.split('_')
    p_n=pp[0]
    bin_indx=np.int(pp[1])
    zbins[bin_indx][p_n]=p_value
    return zbins

In [29]:
def get_model(params,data,kappa0,z_bins,log_prior,indx):
#     t1=time.time()
#     print('get_model',params,log_prior)
    if not np.isfinite(log_prior):
        return np.zeros_like(data)
#     kappa=copy.deepcopy(kappa0)
    kappa=kappa0
    cosmo_params=copy.deepcopy(cosmo_fid)
    Ang_PS=copy.deepcopy(kappa.Ang_PS)
    zbins=copy.deepcopy(z_bins)
#     t2=time.time()
    i=0
    for p in params_order:
        if cosmo_params.get(p) is not None:
            cosmo_params[p]=params[i]
        else:
            zbins=assign_zparams(zbins=zbins,p_name=p,p_value=params[i])
        i+=1
    zbins={'galaxy':zbins,'shear':zbins}
#     t3=time.time()
    model=kappa.tomo_short(cosmo_params=cosmo_params,z_bins=zbins,Ang_PS=Ang_PS)
#     print('get_model',indx,time.time()-t1,t2-t1,t3-t2)
    return model

In [30]:
def chi_sq(params,data,cov_inv,kappa0,z_bins):
    t1=time.time()
    params=np.atleast_2d(params)
    log_priors=get_priors(params)
#     print('params shape',params.shape)
    n_params=len(params)
#     models=client.map(get_model,params)
#     print(models)
#     models=client.gather(models)
#     print(models.shape)
    models={}
    for i in np.arange(n_params):
        models[i]=delayed(get_model)(params[i],data,kappa0,z_bins,log_priors[i],i)
    models=client.compute(models).result()
    models=np.array(list(models.values())) #fixme: check to ensure ordering
    loss=data-models
    chisq=np.zeros(n_params)-np.inf
    for i in np.arange(n_params):
      chisq[i]=-0.5*loss[i]@cov_inv@loss[i].T
#     chisq=-0.5*loss@(cov_inv@loss.T)
#     chisq=np.diag(chisq) #fixme: check to ensure ordering
    chisq+=log_priors
    print('chisq',time.time()-t1)
    return chisq

In [31]:
#for student-t, not guaranteed to work at df<3
class Sample_dist():
    def __init__(self,mean,cov,dist='norm',ndim=2,samples_uniform=None,dist_kwargs={},dist_args=[],df=3,grid='Sobol',N_points=512**2):
        self.__dict__.update(locals())
        self.dist1d=getattr(scipy.stats,dist)
        if dist=='t':
            x=self.samples_uniform==0
            if np.any(x):
                print('moving 0 in uniform sample to 1.e-20 to avoid -np.inf')
                self.samples_uniform[x]=1.e-20
        if samples_uniform is None:
            self.grid=getattr(qmcpy,grid)
            self.grid=self.grid(dimension=ndim,randomize=True) 
            self.samples_uniform=self.grid.gen_samples(N_points)
        self.cov_diag()
#         self.samples_transform()
    def cov_diag(self):
        self.eig_val,self.eig_vec=np.linalg.eig(self.cov)
        self.eig_vec_inv=np.linalg.inv(self.eig_vec)
        self.cov_diag=self.eig_vec.T@self.cov@self.eig_vec
        self.err_diag=np.sqrt(np.diag(self.cov_diag))
        
    def samples_transform_norm(self):
        samples_trans=np.zeros_like(self.samples_uniform)
        for i in np.arange(self.ndim):
            samples_trans[:,i]=scipy.stats.norm.ppf(self.samples_uniform[:,i],loc=0,scale=self.err_diag[i])#,**self.dist_kwargs)
        samples_trans=samples_trans@self.eig_vec_inv #FIXME: some problem with t dist at df<=2 (covariance is undefined)
        samples_trans+=self.mean
        self.samples_trans=samples_trans
    
    def samples_transform_t2(self):#appears not to work well at low df with non-diag covariance, though pdf test is passed
                                    #problem is small errors in ppf function, primarily due to numerical errors in numpy.
                                    #mpmath helps, but is very slow.
        """
                pv=np.linspace(-10**4,10**4,100)
                qv=scipy.stats.t.cdf(pv,df=30,loc=0,scale=1)
                pv2=scipy.stats.t.ppf(qv,df=30,loc=0,scale=1)
                np.all(np.isclose(pv,pv2)),np.all(pv==pv2)
                np.all(np.isclose(np.tan(np.arctan(pv)),pv)) #this passes
                np.all(np.tan(np.arctan(pv))==pv)#this fails
        """
        samples_trans=np.zeros_like(self.samples_uniform)
        for i in np.arange(self.ndim):
            samples_trans[:,i]=student_t_ppf(self.samples_uniform[:,i],loc=0,scale=self.err_diag[i],df=self.df)
        self.prob_t2=scipy.stats.multivariate_t.pdf(samples_trans,loc=self.mean*0,df=self.df,shape=self.cov_diag)
        
        samples_trans=samples_trans@self.eig_vec_inv #FIXME: some problem with t dist at df<=2 (covariance is undefined)
        samples_trans+=self.mean
        
        self.samples_trans_t2=samples_trans
        self.prob_t22=scipy.stats.multivariate_t.pdf(samples_trans,loc=self.mean,df=self.df,shape=self.cov)
        self.prob_norm_t2=scipy.stats.multivariate_normal.pdf(samples_trans,mean=self.mean,cov=self.cov)
        print('samples_transform_t2, pdf test:',np.all(np.isclose(self.prob_t2,self.prob_t22)), np.all(self.prob_t2==self.prob_t22))

    def samples_transform_t(self):
        self.samples_transform_norm()
        
        self.chi2=scipy.stats.chi2.ppf(self.samples_uniform[:,0],df=self.df,loc=0,scale=1)
        self.chi2/=self.df
        
        self.samples_trans-=self.mean
        self.samples_trans/=np.random.permutation(np.sqrt(self.chi2[:,None]))
        self.samples_trans+=self.mean
        
        self.prob=scipy.stats.multivariate_t.pdf(self.samples_trans,loc=self.mean,df=self.df,shape=self.cov)
        self.prob_norm=scipy.stats.multivariate_normal.pdf(self.samples_trans,mean=self.mean,cov=self.cov)

In [32]:
def get_samples(sample_dist='t',N_points=512**2,grid='Sobol'):
    ndim=len(params_order)
    p0=priors_mean
    cov=priors_cov
    sd=Sample_dist(grid=grid,N_points=N_points,dist=sample_dist,mean=priors_mean,cov=cov,ndim=ndim)
    if sample_dist=='t':
        sd.samples_transform_t()
    if sample_dist=='norm':
        sd.samples_transform_norm()
    return sd

In [33]:
N_points=512
grid='Sobol'
sample_dist='t'

In [34]:
ncpu=25

In [38]:
def sample_params(fname=''):
    Sample_dist=get_samples(N_points=N_points,grid=grid,sample_dist=sample_dist)
    chisq_wts=chi_sq(Sample_dist.samples_trans,data,cov_inv,kappa0,zs_bin1)
    outp={}
    outp['chain']=Sample_dist.samples_trans
    outp['p0']=Sample_dist.prob
    outp['params']=params_order
    outp['ln_prob']=chisq_wts
    return outp

In [39]:
kappa0.Ang_PS.reset()
# kappa0.Ang_PS.angular_power_z()

In [40]:
outp=sample_params()

chisq 300.45402455329895


In [ ]:
outp['l0']=l0
outp['l_bins']=l_bins
outp['do_xi']=do_xi
outp['do_pseudo_cl']=do_pseudo_cl
outp['use_binned_l']=use_binned_l
outp['use_binned_theta']=use_binned_theta
outp['data']=data
outp['zbins']=zs_bin1
outp['cov_inv']=cov_inv
outp['params_order']=params_order


In [ ]:
file_home='/verafs/scratch/phy200040p/sukhdeep/physics2/skylens/tests/imaster/'
if do_xi:
    fname_out='xi_{nz}_bl{bl}_bth{bth}_nw{nw}_ns{ns}.pkl'.format(nz=zs_bin1['nbins'],bl=np.int(use_binned_l),bth=np.int(use_binned_theta),ns=nsteps,nw=nwalkers)
if do_pseudo_cl:
    fname_out='pcl_{nz}_bl_bth.pkl'.format(nz=zs_bin1['nbins'],bl=np.int(use_binned_l),bth=np.int(use_binned_theta))
fname_out=file_home+fname_out
with open(fname_out, 'wb') as f:
    pickle.dump(outp,f)

In [ ]:
from skylens import *
PS=Power_Spectra()

In [ ]:
z1=np.linspace(.1,1,10)
%time PS.pk_func(z=z1)

In [ ]:
z1=np.linspace(.1,1,100)
%time PS.pk_func(z=z1)